## Lab 10. Face detection

## Lab tasks

#### Apply Viola Jones Algorithm to detect human faces in the image. The result should look like this:
![](https://i.imgur.com/r9wLclq.png)

#### Apply Viola Jones Algorithm to detect cat faces in the image. The result should look like this:
![](https://i.imgur.com/nSLRiKb.png)

#### Replace human faces with cat faces:
![](https://i.imgur.com/SIRQi6M.png)

#### Apply deep learning to face detection:
![](https://i.imgur.com/qQZoXvN.png)

#### Apply any of these algorithms to detect faces on each frame of the video and if there are more than two faces swap them between each other. You will get something like this but on a video:
![](https://i.imgur.com/shTuXvS.png)


## Algorithms for face detection

### Viola Jones using OpenCV
```
# use haarcascade_frontalface_default.xml to detect people's faces
# use haarcascade_frontalcatface.xml to detect cat's faces

  face_cascade_pulp = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
  print(face_cascade_pulp)
  start = time.time()
  faces = face_cascade_pulp.detectMultiScale(img_pulp)
  print(f'time taken: {time.time()-start}')
  print(len(faces))
  for (x,y,w,h) in faces:
      cv2.rectangle(img_pulp_copy,(x,y),(x+w,y+h),(255,0,0),4)
          
  showInRow([img_pulp_copy])
```
Viola Jones explained: [link](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html)

### Face Detection in OpenCV Using Deep Learning
Using a pretrained neural network
```
  net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')
  confidence = 0.5

  (h, w) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB = False)
      
  net.setInput(blob)
  detections = net.forward()

  # filter out weak detections
  for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > 0.5:
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")

      cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
```

## Import libaries

In [ ]:
import cv2
import time
import numpy as np
from IPython.display import clear_output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
import time, cv2, math
from typing import AnyStr, Any, Callable
%matplotlib inline

### Download images

In [ ]:
!wget https://avatars.mds.yandex.net/get-kinopoisk-post-img/1642096/6a983456f20bd5c47db18643146cd8e7/960x540 -O pulp.jpg
!wget https://vignette.wikia.nocookie.net/houseofnight/images/8/8b/Cats.jpg/revision/latest?cb=20130812053537 -O cats.jpg
!wget https://github.com/RufinaMay/CV2019Fall_Pictures/raw/d0c95c6b3ed54dbb1c6eb7117a8202357617af24/okgo2.mp4?raw=true -O vid.mp4
clear_output()
print('Download completed!')

### Helper functions

In [ ]:
# def read_and_resize_image(filename, grayscale = False, fx= 0.5, fy=0.5):
#   if grayscale:
#     img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
#   else:
#     imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
#     img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
#   img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)
#   return img_result


def read_and_resize(filename: str, grayscale: bool = False, fx: float = 1.0, fy: float = 1.0):
    if grayscale:
      img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
      imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
      # convert to rgb
      img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    # resize
    if fx != 1.0 and fy != 1.0:
      img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)
    return img_result


def showInRow(list_of_images, titles = None, disable_ticks = False):
  count = len(list_of_images)
  for idx in range(count):
    subplot = plt.subplot(1, count, idx+1)
    if titles is not None:
      subplot.set_title(titles[idx])

    img = list_of_images[idx]
    cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
    subplot.imshow(img, cmap=cmap)
    if disable_ticks:
      plt.xticks([]), plt.yticks([])
  plt.show()



def process_video(video_path, frame_process):
  vid = cv2.VideoCapture(video_path)
  try:
    while(True):
      ret, frame = vid.read()
      if not ret:
        vid.release()
        break

      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      print(frame_process)
      if frame_process is not None:
        print("processing")
        frame = frame_process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(frame)

  except KeyboardInterrupt:
    vid.release()


## Perform Viola Jones algorithm

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalcatface.xml
clear_output()
print('Download completed!')

## Open and display image

In [ ]:
img_pulp = read_and_resize('pulp.jpg', grayscale = False, fx=1, fy=1)
showInRow([img_pulp])

## Find faces and draw rectangles around them

In [ ]:
img_pulp_copy = img_pulp.copy()


face_cascade_pulp = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
print(face_cascade_pulp)
start = time.time()
faces = face_cascade_pulp.detectMultiScale(img_pulp)
print(f'time taken: {time.time()-start}')
print(len(faces))
for (x,y,w,h) in faces:
    cv2.rectangle(img_pulp_copy,(x,y),(x+w,y+h),(0,0,255),4)

showInRow([img_pulp_copy])

## Same for cats

In [ ]:
img_cats = read_and_resize('cats.jpg', grayscale = False, fx= 0.5, fy=0.5)
showInRow([img_cats])

In [ ]:
face_cascade_cats = cv2.CascadeClassifier('haarcascade_frontalcatface.xml')

cat_faces_positions = face_cascade_cats.detectMultiScale(img_cats)
print(len(cat_faces_positions))
cat_faces = []
for (x,y,w,h) in cat_faces_positions:
  cat_faces.append(img_cats.copy()[y:y+h, x:x+w])
  cv2.rectangle(img_cats,(x,y),(x+w,y+h),(255,0,0),4)
showInRow([img_cats])
showInRow(cat_faces)

## Swap human faces with cat faces

In [ ]:
img_copy_pulp = img_pulp.copy()
i=0
for (x,y,w,h) in faces:
    fs = cv2.resize(cat_faces[i],(w,h))
    img_copy_pulp[y:y+h, x:x+w] = fs
    i+=1

showInRow([img_copy_pulp])

## Perform Neural Network Algorithm

In [ ]:
# Upload weights and parameters

!gdown 1pOilaivGeUTE5mCxDZm0rEwXp0aAS494
!gdown 16jQB-lRs32cxFN5URulhOd1_3qc3F3UX

In [ ]:
class face_detector():
  def __init__(self):
    self.net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', '/content/res10_300x300_ssd_iter_140000.caffemodel')
    self.confidence = 0.5

  def forward(self, image):
    (self.h, self.w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB = False)

    self.net.setInput(blob)
    self.detections = self.net.forward()

  def detect(self, image):
    self.forward(image)
    for i in range(self.detections.shape[2]):
      confidence = self.detections[0, 0, i, 2]

      if confidence > 0.5:
        box = self.detections[0, 0, i, 3:7] * np.array([self.w, self.h, self.w, self.h])
        (startX, startY, endX, endY) = box.astype("int")

        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)

    return image


In [ ]:
img_pulp = read_and_resize_image('pulp.jpg', grayscale = False, fx= 1, fy=1)

FD = face_detector()

showInRow([FD.detect(img_pulp)])

## Face detection on the video

In [ ]:
class ViolaJones():
  def __init__(self):
    self.face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

  def detect(self, image):
    img_copy = image.copy()
    # gray = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
    # Detect faces
    faces = self.face_cascade.detectMultiScale(img_copy, scaleFactor=1.0, minNeighbors=1, minSize=(10, 10))
    # faces = self.face_cascade.detectMultiScale(image)
    print(len(faces))
    if len(faces)>0:
      for (x,y,w,h) in faces:
        img_copy = cv2.rectangle(img_copy,(x,y),(x+w,y+h),(255,0,0),4)
        print('ffff')
        print(x,y,w,h)
        print(';;;')

    return img_copy
#       cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),4)

In [ ]:
# FD = face_detector()
VJ = ViolaJones()

def detect_faces(frame):
  return VJ.detect(frame)
  # return FD.detect(frame)

In [ ]:
# !gdown 1Wph24YR8r2fOm0oSb42SvK3_mosHf2IA

In [ ]:
frame_width,frame_height = 1280, 720
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (frame_height, frame_width))

# You have to define a function detect_faces(frame) that takes a frame, does the processing and returns the output frame!
# process_video("vid.mp4", detect_faces)
vid = cv2.VideoCapture("vid.mp4")
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
try:
  while(True):
    ret, frame = vid.read()
    if not ret:
      vid.release()
      break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (0,0), fx=1.0, fy=1.0, interpolation = cv2.INTER_LINEAR)
    img_copy = frame.copy()
    faces = face_cascade.detectMultiScale(img_copy)
    print(len(faces))
    if len(faces)>0:
      for (x,y,w,h) in faces:
        img_copy = cv2.rectangle(img_copy,(x,y),(x+w,y+h),(255,0,0),4)
        print(x,y,w,h)
    showInRow([img_copy])
    frame = cv2.cvtColor(img_copy, cv2.COLOR_RGB2BGR)
    out.write(frame)

except KeyboardInterrupt:
  vid.release()

out.release()

### **Hough Transform**

## HoughLines

**cv2.HoughLines(**)
returns an array of (rho, theta) values. rho is measured in pixels and theta is measured in radians.

First parameter, Input image should be a binary image (so apply threshold or use canny edge detection before finding applying hough transform).

Second and third parameters are rho and theta accuracies respectively.

Fourth argument is the threshold, which means minimum vote it should get for it to be considered as a line.

cv2.HoughLines(edges,1,np.pi/180,200)


------------------------------------------------

**cv2.HoughLinesP()**

is an optimization of Hough Transform

returns the two endpoints of lines

lines = cv2.HoughLinesP(E,rho = 1,theta = 1*np.pi/180,threshold = 100,minLineLength = 100,maxLineGap = 50)

In [ ]:
!wget "https://www.dropbox.com/s/vte15iohli93z5j/road.jpg?dl=0" -O road.jpg

In [ ]:
img = read_and_resize_image("road.jpg")
showInRow([img])

In [ ]:
def detectLines(max_slider):
  global img
  global dst
  global gray

  dst = np.copy(img)

  th1 = 700
  th2 = 200
  edges = cv2.Canny(img, th1, th2)
  showInRow([edges])
	# Apply probabilistic hough line transform
  lines = cv2.HoughLinesP(edges, 2, np.pi/180.0, 50, minLineLength=10, maxLineGap=100)
  # show_in_row([dst])
	# Draw lines on the detected points
  for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(dst, (x1, y1), (x2, y2), (0,0,255), 1)



# Create a copy for later usage
dst = np.copy(img)

# Convert image to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Initialize threshold value
initThresh = 500

# Maximum threshold value
maxThresh = 1000

# cv2.createTrackbar("threshold", "Result Image", initThresh, maxThresh, onTrackbarChange)
detectLines(initThresh)
showInRow([dst])

### **HoughCircles**

The function HoughCircles is used in OpenCV to detect the circles in an image. It takes the following parameters:
  
  image: The input image.

  method: Detection method, The available methods are HOUGH_GRADIENT and HOUGH_GRADIENT_ALT.

  dp: the Inverse ratio of accumulator resolution and image resolution.

  mindst: minimum distance between centers od detected circles.

  param_1 and param_2: These are method specific parameters.

  min_Radius: minimum radius of the circle to be detected.

  max_Radius: maximum radius to be detected.

https://docs.opencv.org/4.5.1/dd/d1a/group__imgproc__feature.html#ga47849c3be0d0406ad3ca45db65a25d2d

HoughCircles function has **inbuilt** canny detection, therefore it is not required to detect edges explicitly in it.

In [ ]:
!wget "https://www.dropbox.com/s/tvqy4aq9ts9po04/eyes.jpg?dl=0" -O eyes.jpg

In [ ]:
def detectCircle(max_slider):
    cimg = np.copy(img)

    p1 = max_slider
    p2 = max_slider * 0.4

    # Detect circles using HoughCircles transform
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, cimg.shape[0]/64, param1=p1, param2=p2, minRadius=25, maxRadius=50)

    # If at least 1 circle is detected
    if circles is not None:
        cir_len = circles.shape[1] # store length of circles found
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the outer circle
            cv2.circle(cimg, (i[0], i[1]), i[2], (0, 255, 0), 2)
    else:
        cir_len = 0 # no circles detected

    # Display output image
    showInRow([cimg])





# Read image
img = cv2.imread('eyes.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB )
showInRow([img])
# Convert to gray-scale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Trackbar will be used for changing threshold for edge
initThresh = 105
maxThresh = 200
detectCircle(initThresh)

## Exercise

In [ ]:
!wget "https://www.dropbox.com/s/z40h2aeybcsznln/paper_test.jpg?dl=0" -O paper_test.jpg

In [ ]:
img = read_and_resize("paper_test.jpg")
showInRow([img])

In [ ]:
#TODO
# Find the paper's corners

def find_paper_corners(color_img: np.array) -> np.array:
  # return np.array with shape (4, 2)
  # order: [top_left, top_right, bot_right, bot_left]

  gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
  gray_img = cv2.GaussianBlur(gray_img, None, 1.0)
  th, gray_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_OTSU)

  cnts, hierarchy = cv2.findContours(gray_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

  paper_contour = None
  for c in cnts:
      peri = cv2.arcLength(c, True)
      approx = cv2.approxPolyDP(c, 0.1 * peri, True)

      if len(approx) == 4:
        paper_contour = approx
        break

  img_copy = color_img.copy()
  if paper_contour is not None:
    cv2.drawContours(img_copy, [paper_contour], -1, (0, 255, 0), 2)

  showInRow([gray_img, img_copy])
  paper_contour = paper_contour.reshape((4, 2))
  paper_contour = paper_contour[::-1]
  paper_contour[:2], paper_contour[2:] = paper_contour[2:].copy(), paper_contour[:2].copy()
  return paper_contour.reshape((4, 2))

corners = find_paper_corners(img)
print(corners)
if corners.shape == (4, 2):
  print("You are good")
  img_copy = img.copy()
  cv2.polylines(img_copy, [corners], True, (255,0,0), 3)
  showInRow([img_copy])
else:
  print("Shape must contain the answer to the ultimate question of life")


In [ ]:
# TODO

def undistort_perspective(img: np.array, corners: np.array) -> np.array:
  dst_size = (1120, 1584)
  dst_points = np.array([[0, 0], [dst_size[0], 0], [dst_size[0], dst_size[1]], [0, dst_size[1]]], np.int32)
  h, status = cv2.findHomography(corners, dst_points)
  return cv2.warpPerspective(img, h, dst_size)

paper = undistort_perspective(img, corners)
showInRow([paper])

In [ ]:
def find_answers(color_img: np.array) -> list:
  # return in format [[A], [B, C], [], [A], ..]
  gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
  # cv2.Canny()
  dst = cv2.Canny(gray_img, 200, 200, None, 3)
  # _, dst = cv2.threshold(gray_img, 160, 255, cv2.THRESH_BINARY_INV)
  showInRow([dst])

  lines = cv2.HoughLines(dst, 3, np.pi / 90, 500)
  lines = sorted(lines, key=lambda x: x[0][0])
  img_copy = color_img.copy()

  i1, i2 = 0, 0
  xs, ys = [], []
  if lines is not None:
    for i in range(len(lines)):
      rho = lines[i][0][0]
      theta = lines[i][0][1]
      a = math.cos(theta)
      b = math.sin(theta)
      x0 = a * rho
      y0 = b * rho
      pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
      pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
      ang = theta * 180 / math.pi
      if ang > 45:
        i1 += 1
        cv2.line(img_copy, pt1, pt2, (i1*30,0,255), 3, cv2.LINE_AA)
        ys.append(pt1[1])
      else:
        i2 += 1
        cv2.line(img_copy, pt1, pt2, (i2*60,255,0), 3, cv2.LINE_AA)
        xs.append(pt1[0])
  showInRow([color_img, img_copy])

  answers = []
  column_names = ['A', 'B', 'C', 'D']
  thresh = 50
  for row in range(len(ys)-1):
    answers.append([])
    for col in range(len(xs)-1):
      content = dst[ys[row]+12:ys[row+1]-4, xs[col]+10:xs[col+1]-10]
      if np.count_nonzero(content) > thresh:
        answers[row].append(column_names[col])
  return answers

answers = find_answers(paper[150:-100,100:-100])
print(answers)
if len(answers) == 20 and answers[0] == ['C']:
  print("Seems like you did well!")
else:
  print("TODO")